In [1]:
import pandas as pd

In [56]:
cols = ['RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription', 'spn', 'fmi', 'active', 'activeTransitionCount',
           'EquipmentID', 'Latitude', 'Longitude']

faults = pd.read_csv("../data/J1939Faults.csv", usecols = cols, dtype = str)
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851
1,2,990360,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.42125,-87.767361
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361
4,5,990416,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638


In [57]:
faults.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187335 entries, 0 to 1187334
Data columns (total 11 columns):
 #   Column                 Non-Null Count    Dtype 
---  ------                 --------------    ----- 
 0   RecordID               1187335 non-null  object
 1   ESS_Id                 1187335 non-null  object
 2   EventTimeStamp         1187335 non-null  object
 3   eventDescription       1126490 non-null  object
 4   spn                    1187335 non-null  object
 5   fmi                    1187335 non-null  object
 6   active                 1187335 non-null  object
 7   activeTransitionCount  1187335 non-null  object
 8   EquipmentID            1187335 non-null  object
 9   Latitude               1187335 non-null  object
 10  Longitude              1187335 non-null  object
dtypes: object(11)
memory usage: 99.6+ MB


In [58]:
# convert dtypes as needed
# some columns have numeric-appearing values but they're not continuous variables, leaving them as strings
faults['EventTimeStamp'] = pd.to_datetime(faults['EventTimeStamp'])
faults['activeTransitionCount'] = faults['activeTransitionCount'].astype(int)

In [59]:
# time of day for faults could be interesting, split event column to date and timestamp columns
faults['event_date'] = faults['EventTimeStamp'].dt.date
faults['event_time'] = faults['EventTimeStamp'].dt.time

In [60]:
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,event_date,event_time
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,2015-02-21,10:47:13
1,2,990360,2015-02-21 11:34:34,NaN,629,12,True,127,1439,38.857638,-84.626851,2015-02-21,11:34:34
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.42125,-87.767361,2015-02-21,11:35:31
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21,11:35:33
4,5,990416,2015-02-21 11:39:41,NaN,4364,17,False,2,1674,38.416481,-89.442638,2015-02-21,11:39:41


In [77]:
# mask for each service station, lat and long to 2 decimal places
station_1 = (faults['Latitude'].str.contains('36.06')) & (faults['Longitude'].str.contains('86.43'))
station_2 = (faults['Latitude'].str.contains('35.58')) & (faults['Longitude'].str.contains('86.44'))
station_3 = (faults['Latitude'].str.contains('36.19')) & (faults['Longitude'].str.contains('83.17'))

# dataframe without faults associated with service locations
faults_nonservice = faults[~(station_1 | station_2 | station_3)]

### next steps
- add date and time breakout columns (year, month, date, weekday, hour, day segment (divide 24 hours into quadrants))
- merge diagnostic data with filtered faults data (inner join)
- figure out imputing methods
- for models, imput after train/test split
- do some eda on the data

In [78]:
faults_nonservice = faults_nonservice.copy()

faults_nonservice['event_year'] = faults_nonservice['EventTimeStamp'].dt.year
faults_nonservice['event_month'] = faults_nonservice['EventTimeStamp'].dt.month
faults_nonservice['event_day'] = faults_nonservice['EventTimeStamp'].dt.day
faults_nonservice['event_dayofweek'] = faults_nonservice['EventTimeStamp'].dt.dayofweek
faults_nonservice['event_dayname'] = faults_nonservice['EventTimeStamp'].dt.day_name()
faults_nonservice['event_hour'] = faults_nonservice['EventTimeStamp'].dt.hour
faults_nonservice['event_time_quadrant'] = faults_nonservice['EventTimeStamp'].dt.hour // 4

In [79]:
faults_nonservice.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,event_date,event_time,event_year,event_month,event_day,event_dayofweek,event_dayname,event_hour,event_time_quadrant
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,2015-02-21,10:47:13,2015,2,21,5,Saturday,10,2
1,2,990360,2015-02-21 11:34:34,NaN,629,12,True,127,1439,38.857638,-84.626851,2015-02-21,11:34:34,2015,2,21,5,Saturday,11,2
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.42125,-87.767361,2015-02-21,11:35:31,2015,2,21,5,Saturday,11,2
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,2015-02-21,11:35:33,2015,2,21,5,Saturday,11,2
4,5,990416,2015-02-21 11:39:41,NaN,4364,17,False,2,1674,38.416481,-89.442638,2015-02-21,11:39:41,2015,2,21,5,Saturday,11,2
